In [1]:
from datetime import datetime
import functools
from IPython.display import HTML
import jax
from jax import numpy as jp
import numpy as np
from typing import Any, Dict, Sequence, Tuple, Union
from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
from brax.training.agents.apg import train as apg
from brax.training.agents.apg import networks as apg_networks
from brax.io import html, mjcf, model
from etils import epath
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from ml_collections import config_dict
import mujoco
from mujoco import mjx
from jax import vmap
import jax.random
from jax import lax

In [2]:
from utils.SimpleConverter import SimpleConverter
from agent_mimic_env.agent_template import HumanoidTemplate
from agent_mimic_env.agent_eval_template import HumanoidEvalTemplate
from agent_mimic_env.agent_test_apg import HumanoidAPGTest
from utils.util_data import *
from copy import deepcopy

In [3]:
import agent_mimic_env
from agent_mimic_env import register_mimic_env


In [4]:

import distutils.util
import os
xla_flags = os.environ.get('XLA_FLAGS', '')
xla_flags += ' --xla_gpu_triton_gemm_any=True'
os.environ['XLA_FLAGS'] = xla_flags
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

Setting environment variable to use GPU rendering:
env: MUJOCO_GL=egl


In [5]:
import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.8" # 0.9 causes too much lag. 
from datetime import datetime
import functools

# Math
import jax.numpy as jp
import numpy as np
import jax
from jax import config # Analytical gradients work much better with double precision.
config.update("jax_debug_nans", True)
config.update("jax_enable_x64", True)
config.update('jax_default_matmul_precision', jax.lax.Precision.HIGH)
from brax import math

# Sim
import mujoco
import mujoco.mjx as mjx

# Brax
from brax import envs
from brax.base import Motion, Transform
from brax.io import mjcf
from brax.envs.base import PipelineEnv, State
from brax.mjx.pipeline import _reformat_contact
from brax.training.acme import running_statistics
from brax.io import model

# Algorithms
# from brax.training.agents.apg import train as apg
# from brax.training.agents.apg import networks as apg_networks
from brax.training.agents.ppo import train as ppo

# Supporting
from etils import epath
import mediapy as media
import matplotlib.pyplot as plt
from ml_collections import config_dict
from typing import Any, Dict

In [6]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [7]:
import yaml
from box import Box
# Path to your YAML file
yaml_file_path = 'config_params/punch2.yaml'
# Load the YAML file
with open(yaml_file_path, 'r') as file:
    args = Box(yaml.safe_load(file))

In [8]:
print(args)

{'num_envs': 256, 'num_eval_envs': 32, 'lr': 0.0003, 'max_it': 1000, 'max_grad_norm': 0.4, 'seed': 0, 'system_config': 'humanoid', 'demo_replay_mode': 'threshold', 'threshold': 0.4, 'normalize_observations': True, 'cycle_len': 130, 'ep_len': 65, 'ep_len_eval': 54, 'use_lr_scheduler': True, 'reward_scaling': 0.02, 'rot_weight': 0.5, 'vel_weight': 0.01, 'ang_weight': 0.01, 'deep_mimic_reward_weights': {'w_p': 0.3, 'w_v': 0.1, 'w_e': 0.5, 'w_c': 0.1}, 'deep_mimic_weights_factors': {'w_pose': 2, 'w_angular': 0.005, 'w_efector': 5, 'w_com': 100}, 'model': 'models/final_humanoid2.xml', 'ref': 'motions/humanoid3d_punch_duplicated.txt'}


In [9]:
args.deep_mimic_reward_weights.w_p

0.3

In [10]:
from agent_mimic_env.pds_controllers_agents import feedback_pd_controller, stable_pd_controller_action


In [11]:
def generate_env_apg_train(trajectory,model_path,env_name_apg,args):
    
    #model_path = 'anybotics_anymal_c/scene_mjx.xml'
    #model_path = 'models/humanoidMjx.xml'
    model_path = 'models/other_humanoid2.xml'
    #model_path = args.model
    
    env_kwargs = dict(referece_data=trajectory,model_path=model_path,args=args) 
    
    env = envs.get_environment(env_name_apg,**env_kwargs)
    return env
    

In [12]:
trajectory = SimpleConverter(args.ref)
trajectory.load_mocap()
    #model_path = 'models/final_humanoid.xml'
model_path = args.model
envs.register_environment('humanoidApgTest',HumanoidAPGTest)
env_name_apg = 'humanoidApgTest'

env_apg = generate_env_apg_train(trajectory,model_path,env_name_apg,args)

env_apg.set_pd_callback(stable_pd_controller_action)

jit_reset = jax.jit(env_apg .reset)
jit_step = jax.jit(env_apg .step)


In [13]:
env_apg.sys.opt

Option(timestep=0.002, impratio=Array(100., dtype=float64, weak_type=True), tolerance=Array(1.e-08, dtype=float64, weak_type=True), ls_tolerance=Array(0.01, dtype=float64, weak_type=True), gravity=Array([ 0.  ,  0.  , -9.81], dtype=float64), wind=Array([0., 0., 0.], dtype=float64), density=Array(0., dtype=float64, weak_type=True), viscosity=Array(0., dtype=float64, weak_type=True), has_fluid_params=False, integrator=<IntegratorType.EULER: 0>, cone=<ConeType.PYRAMIDAL: 0>, jacobian=<JacobianType.AUTO: 2>, solver=<SolverType.NEWTON: 2>, iterations=1, ls_iterations=4, disableflags=<DisableBit.EULERDAMP: 16384>)

In [14]:
print(env_apg.sys.opt.timestep)
print(env_apg.dt)

0.002
0.02


In [15]:

print(env_apg.rollout_lenght)
print(env_apg.cycle_len)
print(env_apg.rot_weight)
print(env_apg.vel_weight)
print(env_apg.ang_weight)
print(env_apg.reward_scaling)
print(env_apg.dt)


65
130
0.5
0.01
0.01
0.02
0.02


In [16]:
# initialize the state

#for the replay
state = jit_reset(jax.random.PRNGKey(2))
rollout = [state.pipeline_state]



# grab a 500 steps
n_steps = 500

In [17]:
print(state.metrics)
print(state.info)

{'fall': Array(0., dtype=float64), 'pose_error': Array(0., dtype=float64), 'reference_angular': Array(0., dtype=float64, weak_type=True), 'reference_position': Array(0., dtype=float64, weak_type=True), 'reference_rotation': Array(0., dtype=float64, weak_type=True), 'reference_velocity': Array(0., dtype=float64, weak_type=True), 'step_index': Array(60., dtype=float64)}
{'default_pos': Array([ 1.18543717,  1.27664797,  0.790196  ,  0.97201826, -0.01898119,
        0.04365162,  0.23003207, -0.09011222,  0.31782795, -0.04773092,
        0.26030649, -0.12344302, -0.44215609, -0.37697787,  0.02596283,
       -0.18623492,  2.13618706,  0.73544372,  0.36881986, -0.80315166,
        1.98275626,  0.03526071, -0.59545573, -0.01541848, -1.10715834,
       -0.05758162, -0.31225456, -0.07258391,  0.39222261, -0.63284242,
        0.19183088, -1.16367734,  0.15139849,  0.08802346,  0.13914152],      dtype=float64), 'kinematic_ref': Array([ 1.18543717,  1.27664797,  0.790196  ,  0.97201826, -0.01898119

In [18]:
for i in range(env_apg .rollout_lenght-1):
    
    #print(i)
 
    ctrl = -0.1 * jp.ones(env_apg .sys.nu)
   
  
         
    state = jit_step(state, ctrl)
  
    
    #print(state.pipeline_state.geom_xpos)
    #if state.done:
     #   print(time)
      #  break
    #print("Is done", state.done)
    print("Is rewards", state.reward)
    #print('is step', state.metrics['step_index'])
    #print("reward tuple", state.info['reward_tuple'])
    #print("Is ", state.metrics['pose_error'])
    #print("Is ", state.metrics['reference_angular'])
    #print("Is ", state.metrics['min_reference_tracking'])
    
    rollout.append(state.pipeline_state)

Is rewards -1.3479094790857306e-33
Is rewards -1.6956384453644907e-33
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -2.83134775947142e-33
Is rewards -0.0
Is rewards -7.044508124323149e-33
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -3.168593879803602e-33
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -3.3685142737902365e-33
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards -0.0
Is rewards

In [19]:
from some_math.quaternion_diff import *

In [20]:
HTML(html.render(env_apg.sys.tree_replace({'opt.timestep': env_apg.dt}), rollout))



In [21]:
xd=-2.5 * 3e-3
print(xd)

-0.0075
